Computing Hofstede's Cultural scores (indices).

Aggregating data by question number.

In [2]:
import pandas as pd
import numpy as np

df_responses_gpt = pd.read_csv('responses_parsed.csv', sep=';', encoding='utf-8-sig')
df_responses = df_responses_gpt[['question','nationality','local','score_gpt-4o','score_gpt-3.5-turbo','score_yandexgpt']].groupby(['nationality','question','local']).mean().round(2).reset_index()

Computing indices. Doing remaping to range [0;100].

In [3]:
def map_range(value, from_min, from_max, to_min, to_max):
    value_scaled = (value - from_min) / (from_max - from_min)
    return to_min + (value_scaled * (to_max - to_min))

def calculate_indices(group):
    indices = {}
    
    indices['PDI'] = map_range(
        round(35 * (group.loc[group['question'] == 7, 'score'].values[0] - group.loc[group['question'] == 2, 'score'].values[0]) + 25 * (group.loc[group['question'] == 20, 'score'].values[0] - group.loc[group['question'] == 23, 'score'].values[0]),2)
                               ,-(35+25)*4,(35+25)*4,0,100)
    indices['IDV'] = map_range(
        round(35 * (group.loc[group['question'] == 4, 'score'].values[0] - group.loc[group['question'] == 1, 'score'].values[0]) + 35 * (group.loc[group['question'] == 9, 'score'].values[0] - group.loc[group['question'] == 6, 'score'].values[0]),2)
                               ,-(35+35)*4,(35+35)*4,0,100)
    indices['MAS'] = map_range(
        round(35 * (group.loc[group['question'] == 5, 'score'].values[0] - group.loc[group['question'] == 3, 'score'].values[0]) + 35 * (group.loc[group['question'] == 8, 'score'].values[0] - group.loc[group['question'] == 10, 'score'].values[0]),2)
                               ,-(35+35)*4,(35+35)*4,0,100)
    indices['UAI'] = map_range(
        round(40 * (group.loc[group['question'] == 18, 'score'].values[0] - group.loc[group['question'] == 15, 'score'].values[0]) + 25 * (group.loc[group['question'] == 21, 'score'].values[0] - group.loc[group['question'] == 24, 'score'].values[0]),2)
                               ,-(40+25)*4,(40+25)*4,0,100)
    indices['LTO'] = map_range(
        round(40 * (group.loc[group['question'] == 13, 'score'].values[0] - group.loc[group['question'] == 14, 'score'].values[0]) + 25 * (group.loc[group['question'] == 19, 'score'].values[0] - group.loc[group['question'] == 22, 'score'].values[0]),2)
                               ,-(40+25)*4,(40+25)*4,0,100)
    indices['IVR'] = map_range(
        round(35 * (group.loc[group['question'] == 12, 'score'].values[0] - group.loc[group['question'] == 11, 'score'].values[0]) + 40 * (group.loc[group['question'] == 17, 'score'].values[0] - group.loc[group['question'] == 16, 'score'].values[0]),2)
                               ,-(35+40)*4,(35+40)*4,0,100)
    
    return pd.Series(indices)

models = ['score_gpt-4o', 'score_gpt-3.5-turbo', 'score_yandexgpt']
df_models = {}

for model in models:
    df_model = df_responses[['nationality', 'local', 'question', model]].rename(columns={model: 'score'})
    grouped = df_model.groupby(['nationality', 'local']).apply(calculate_indices).reset_index()
    grouped['model'] = model.replace('score_','')

    df_models[model] = grouped

In [4]:
df_stacked = df_models = pd.concat(df_models.values(), axis=0, ignore_index=True)
cols = df_stacked.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_stacked = df_stacked[cols]

df_stacked.to_csv('responses_indices.csv', sep=';', encoding='utf-8-sig')